In [1]:
#https://keras.io/examples/generative/vae/
#https://keras.io/examples/generative/molecule_generation/

In [16]:
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
import pandas as pd
#================== initialization ==================
LATENT_DIM = 128
VAE_LR = 5e-4
PARQUET_PATH = './data/OptionsEOD.parquet'
SCALER_PATH = './data/scaler.gz'
SCALER_COL  = ['DTE','INTRINSIC_VALUE','C_VOLUME',	'C_BID',	'C_ASK',	'P_BID',	'P_ASK',	'P_VOLUME' ]

In [18]:
#load scaler
scaler = MinMaxScaler()
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    break
scaler = joblib.load(SCALER_PATH)
df_tf = scaler.transform(df[SCALER_COL])

In [24]:
df_tf.shape

(56542, 8)

In [9]:
from src.model import OptionChainGenerator
from src.layer import encoder, decoder

model = OptionChainGenerator(
    encoder(latent_dim = LATENT_DIM, 
            input_shape= , 
            gconv_units = [64, 128], 
            dense_units = [256, 128], 
            dropout_rate= 0.1,
            output_dim  = 
           ), 
    decoder(latent_dim  = LATENT_DIM , 
            output_shape= ,
            dense_units = [128, 256],
            dropout_rate= 0.1
           )
)
model.summery()
vae_optimizer = tf.keras.optimizers.Adam(learning_rate=VAE_LR)
model.compile(vae_optimizer)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - kl_loss: 1.9520 - loss: 45.7004 - reconstruction_loss: 43.7484

ValueError: No loss to compute. Provide a `loss` argument in `compile()`.

In [ ]:
#================== train model ==================
history = model.fit([feature_tensor], epochs=EPOCHS)


In [23]:
import numpy as np

def create_adjacency_matrix(options, threshold=25):
    """
    สร้าง adjacency matrix สำหรับ Options Chain โดยใช้ความใกล้เคียงของ strike price และ DTE
    """
    num_options = len(options)
    adjacency_matrix = np.zeros((num_options, num_options))

    for i in range(num_options):
        for j in range(i, num_options):
            # พิจารณาเชื่อมโยงระหว่าง options หาก strike price ต่างกันไม่เกิน threshold และ DTE เท่ากัน
            if abs(options[i]['strike_price'] - options[j]['strike_price']) <= threshold and options[i]['dte'] == options[j]['dte']:
                adjacency_matrix[i, j] = 1
                adjacency_matrix[j, i] = 1
    
    return adjacency_matrix

options = [
    {'strike_price': 100, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 115, 'dte': 30},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 0, 'dte': 0},  # เปลี่ยนค่าเป็น 0
]

options = [
    {'strike_price': 0, 'dte': 0},
    {'strike_price': 120, 'dte': 30},
    {'strike_price': 150, 'dte': 30},
    {'strike_price': 110, 'dte': 30},
    {'strike_price': 105, 'dte': 30},
    {'strike_price': 100, 'dte': 30},  # เปลี่ยนค่าเป็น 0
]


# ปรับ threshold
threshold = 5

adjacency_matrix = create_adjacency_matrix(options, threshold)
print(adjacency_matrix)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1.]]


In [1]:
import numpy as np
A = np.array(
    [[1,2,3],
    [4,5,6],
    [7,8,9]]
    
)

In [3]:
A[:,:,0]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [4]:
def get_variable_name(variable):
    for name, value in globals().items():
        if value is variable:
            return name
    return None

# Example usage
leo = ['a', 'b']
var_name = get_variable_name(leo)
print(var_name)  # Ou

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])